In [8]:
import numpy as np

data = np.load('mnist.npz')
X_train = data['x_train']
y_train = data['y_train']
X_test = data['x_test']
y_test = data['y_test']

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

def one_hot_encoding(y, num_classes=10):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y] = 1
    return one_hot

y_train_encoded = one_hot_encoding(y_train)
y_test_encoded = one_hote_encoding(y_test)

In [12]:
def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(42)

    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1)) # zero bias

    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    
    return W1, b1, W2, b2

input_size = 784
hidden_size = 128
output_size = 10

W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)

In [13]:
# hidden layer z1 = w1 @ x + b1
# activation 1 a1 = relu(z1)
# output layer z2 = w2 @ a1 + b2
# activation 2 = a2 = softmax(z2)

In [14]:
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

SyntaxError: incomplete input (2551850742.py, line 6)